# Juan Carlos Perez Ramirez
## Procesamiento de Lenguaje Natural
## Examen 2

Para la tarea se propone el uso de una BoW de 50 mil n-gramas (con $n = 1,...,5$, si bien se hicieron pruebas con k-skip-n-gramas que arrojaron peores resultados) construida a partir de la combinacion de los tweets de cada autor en un solo texto (de manera que se tiene un documento por autor), a partir de la cual se entrenan dos Multi Layer Perceptrons, para clasificar el sexo y la nacionalidad del autor, respectivamente. Se probaron diferentes arquitecturas, siendo la presente la mejor de las analizadas.

### Preprocesamiento de datos

In [1]:
import os
import xml.etree.ElementTree as ET
import re

# Rutas de entrada y salida
split = "test"
dir = "es_" + split
path = "../../corpus/examen2/"
truth = path + dir + '/truth_order.txt'  # archivo con los identificadores
xml_dir = path + dir  # carpeta donde están los archivos XML
docs_file = path + split + '.csv'

# abrimos los archivos de salida
with open(docs_file, 'w', encoding='utf-8') as f_textos, \
     open(truth, 'r', encoding='utf-8') as f_in:

    for linea in f_in:
        partes = linea.strip().split(":::")
        if len(partes) != 3:
            continue  # Saltamos líneas mal formateadas

        identificador = partes[0]
        genero = partes[1]
        nacionalidad = partes[2]
        archivo_xml = os.path.join(xml_dir, f"{identificador}.xml")

        if not os.path.exists(archivo_xml):
            print(f"Archivo {archivo_xml} no encontrado, saltando.")
            continue

        try:
            tree = ET.parse(archivo_xml)
            root = tree.getroot()
            documentos = root.find('documents')

            # Guardamos todos los tweets en una lista
            tweets = []
            for doc in documentos.findall('document'):
                texto = doc.text.strip()
                texto = re.sub(r"http\S+|www\S+|https\S+", "", texto)  # elimina URLs
                texto = re.sub(r"@\w+", "<user>", texto) # sustituye menciones por "<user>"
                texto = texto.replace('\n', ' ')  # elimina saltos de línea dentro del tweet
                texto = texto.lower()  # pasar a minúsculas
                tweets.append(texto)

            if tweets:
                linea_usuario = " </s> ".join(tweets) + " </s> " + genero + " </s> " + nacionalidad + "\n"
                f_textos.write(linea_usuario)

        except Exception as e:
            print(f"Error procesando {archivo_xml}: {e}")

In [ ]:
import pandas as pd
def load_data(path_txt):
        texts = []
        genders = []
        nationalities = []

        with open(path_txt, 'r', encoding='utf-8') as f:
            for line in f:
                if "</s>" in line:
                    partes = line.strip().split("</s>")
                    if len(partes) < 3:
                        continue
                    *tweets, genero, nacionalidad = partes
                    texto_completo = " [SEP] ".join(tweets)  # concatena los tweets
                    texts.append(texto_completo)
                    genders.append(genero)
                    nationalities.append(nacionalidad)


        df = pd.DataFrame({'text': texts, 'gender': genders, 'nationality': nationalities})
        gender_map = {' male ': 0, ' female ': 1}
        nat_map = {l: i for i, l in enumerate(sorted(df['nationality'].unique()))}
        df['gender'] = df['gender'].map(gender_map)
        df['nationality'] = df['nationality'].map(nat_map)

        print(f"Nationalities: {nat_map}")

        return df, gender_map, nat_map

# Construccion de BoW

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('spanish')

path_train = "../../corpus/examen2/train.csv"
path_val = "../../corpus/examen2/val.csv"
path_test = "../../corpus/examen2/test.csv"
df, gender_map, nat_map = load_data(path_train)
df_val, _, _ = load_data(path_val)
df_test, _, _ = load_data(path_test)
vectorizer = TfidfVectorizer(max_features=50000, ngram_range=(1, 5))  # TF-IDF con unigramas y bigramas

scaler = StandardScaler(with_mean=False)  # with_mean=False si estás usando sparse matrices (TF-IDF)
X = vectorizer.fit_transform(df['text'])
#X = scaler.fit_transform(X)
X_val = vectorizer.transform(df_val['text'])
#X_val = scaler.transform(X_val)
X_test = vectorizer.transform(df_test['text'])
#X_test = scaler.transform(X_test)

y_gender = df['gender']
y_nat = df['nationality']

y_gender_val = df_val['gender']
y_nat_val = df_val['nationality']

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/juancho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Nationalities: {' argentina': 0, ' chile': 1, ' colombia': 2, ' mexico': 3, ' peru': 4, ' spain': 5, ' venezuela': 6}
Nationalities: {' argentina': 0, ' chile': 1, ' colombia': 2, ' mexico': 3, ' peru': 4, ' spain': 5, ' venezuela': 6}
Nationalities: {' _': 0}


### Entrenamiento de MLP

In [3]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

clf = MLPClassifier(hidden_layer_sizes=(200, 50), max_iter=300, random_state=42)
clf.fit(X, y_gender)

y_pred = clf.predict(X_val)
print("Clasificación de género:\n", classification_report(y_gender_val, y_pred))

Clasificación de género:
               precision    recall  f1-score   support

           0       0.78      0.80      0.79       420
           1       0.79      0.77      0.78       420

    accuracy                           0.79       840
   macro avg       0.79      0.79      0.79       840
weighted avg       0.79      0.79      0.79       840



In [4]:
clf = MLPClassifier(hidden_layer_sizes=(200, 50), max_iter=300, random_state=42)

clf.fit(X, y_nat)

y_pred = clf.predict(X_val)
print("Clasificación de nacionalidad:\n", classification_report(y_nat_val, y_pred))

Clasificación de nacionalidad:
               precision    recall  f1-score   support

           0       0.92      0.96      0.94       120
           1       0.97      0.96      0.97       120
           2       0.92      0.95      0.93       120
           3       0.90      0.90      0.90       120
           4       0.93      0.88      0.91       120
           5       0.86      0.94      0.90       120
           6       0.93      0.83      0.88       120

    accuracy                           0.92       840
   macro avg       0.92      0.92      0.92       840
weighted avg       0.92      0.92      0.92       840



#### Prueba

In [22]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(200, 50), max_iter=300, random_state=42)

clf.fit(X, y_gender)
gen = clf.predict(X_test)

clf.fit(X, y_nat)
nat = clf.predict(X_test)

/home/juancho/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/home/juancho/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [ ]:
split = "test"
dir = "es_" + split
path = "../../corpus/examen2/"
truth = path + dir + '/truth_order.txt'
docs_file = path + split + '.csv'

ids = []
with open(truth, 'r', encoding='utf-8') as f_in:
    for linea in f_in:
        partes = linea.strip().split(":::")
        if len(partes) != 3:
            continue

        ids.append(partes[0])

In [ ]:
predictions = pd.DataFrame({
    'id': ids,
    'gender': gen,
    'nationality': nat
})

inv_gender_map = {v: k for k, v in gender_map.items()}
inv_nat_map = {v: k for k, v in nat_map.items()}
predictions['gender'] = predictions['gender'].map(inv_gender_map)
predictions['nationality'] = predictions['nationality'].map(inv_nat_map)

In [21]:
l = predictions.shape[0]
with open("predictions.csv", "w") as f:
    f.write("Id,Gender:::Nationality\n")
    for i in range(l):
        data = predictions.iloc[i]
        f.write(str(i) + "," + data['gender'].strip() + ':::' + data['nationality'].strip() + "\n")